# <center>DLRM model</center>

### Import depencies

In [6]:
# !pip install fbgemm-gpu
# !pip install fbgemm-gpu-cpu
# !pip install torch
# !pip install torchrec
# !pip install torchrec-cpu
# !pip install pyre_extensions

In [10]:
import pandas as pd
import numpy as np
import random
import torch     
from sklearn.model_selection import train_test_split

### Import Data

In [4]:
df = pd.read_csv(r'/Users/royzalta/Downloads/archive/Books Data with Category Language and Summary/Preprocessed_data.csv')

### train & test split

In [11]:
train_df, test_df = train_test_split(
    df, test_size=0.33, random_state=42)

### Encoding sparse features

In [12]:
cols_dense = ["feat1", "feat2"]
cols_sparse = ['cat1', 'cat2']

In [13]:
def encode(col, map_rev):
    return [map_rev.get(item,item) for item in col]

In [14]:
map_sparse = {}
map_sparse_rev = {}

In [15]:
for feat in cols_sparse:
    map_sparse[feat] = {i: c for i,c in enumerate(train_df[feat].explode().value_counts().index)}
    map_sparse_rev[feat] = {v:k for k, v in map_sparse[feat].items()}

    train_df[feat+'_enc'] = train_df[feat].apply(lambda x: encode(x, map_sparse_rev[feat]))
    test_df[feat+'_enc'] = test_df[feat].apply(lambda x: encode(x, map_sparse_rev[feat]))
    val_df[feat+'_enc'] = test_df[feat].apply(lambda x: encode(x, map_sparse_rev[feat]))

KeyError: 'cat1'